# Cleaning Sheet: TOTALS

In [ ]:
# Load an excel sheet with a specific sheet name to a df
import pandas as pd
data = pd.read_excel('/content/AAdi Compiled Data.xlsx', sheet_name='TOTALS')

In [ ]:
data.head()

In [ ]:
# Rename column Unamed: 1 to Aggregation_Category
data.rename(columns={'Unnamed: 1': 'Demographic_Category',
                     'Period': 'Metric',
                     'Thur 1/20/22 - Tue 4/19/22': '2022',
                     'Thur 1/21/21 - Tue 4/20/21': '2021'}, inplace=True)

In [ ]:
data.head()

In [ ]:
# Convert data to long format
data_long = pd.melt(data, id_vars=['Metric', 'Demographic_Category'], var_name='Year', value_name='Value')

In [ ]:
data_long.head()

In [ ]:
# Check for empty rows
data_long.isnull().sum()

In [ ]:
# Remove empty rows
data_long = data_long.dropna()

# Cleaning Sheet: Traffic

In [ ]:
data_traffic = pd.read_excel('/content/AAdi Compiled Data.xlsx', sheet_name='Traffic')

In [ ]:
data_traffic.head()

In [ ]:
data_traffic.rename(columns={'Unnamed: 0': 'Year'}, inplace=True)

In [ ]:
data_traffic.head()

In [ ]:
# In column year replace the value 'Thur 1/20/22 - Tue 4/19/22' with 2022
data_traffic['Year'] = data_traffic['Year'].replace('Thur 1/20/22 - Tue 4/19/22', 2022)
data_traffic['Year'] = data_traffic['Year'].replace('Thur 1/21/21 - Tue 4/20/21', 2021)

<ipython-input-75-ed4db1d96d29>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_traffic['Year'] = data_traffic['Year'].replace('Thur 1/21/21 - Tue 4/20/21', 2021)


In [ ]:
data_traffic.head()

In [ ]:
# Remove rows where column year is NULL
data_traffic = data_traffic.dropna(subset=['Year'])

In [ ]:
# Convert year column to year data type
data_traffic['Year'] = data_traffic['Year'].astype(int)

In [ ]:
data_traffic.head()

In [ ]:
# Convert data_traffic to long format
data_traffic_long = pd.melt(data_traffic, id_vars=['Year', 'Traffic source'], var_name='Metric', value_name='Value')

In [ ]:
data_traffic_long.head()

In [ ]:
data_long.head()

In [ ]:
data_traffic_long['Traffic source'].value_counts()

In [ ]:
data_long['Demographic_Category'].value_counts()

# Merging the two tables

In [ ]:
# Standardize the 'TOTAL' values
data_long["Demographic_Category"] = data_long["Demographic_Category"].str.upper()
data_traffic_long["Traffic source"] = data_traffic_long["Traffic source"].str.upper()

data_long['Traffic source'] = "TOTAL"
data_traffic_long['Demographic_Category'] = "TOTAL"

# Combine the tables using a union operation
combined_table = pd.concat([data_long, data_traffic_long], ignore_index=True)

In [ ]:
combined_table.head()

In [ ]:
combined_table.info()

In [ ]:
# Convert value column to integer
combined_table['Value'].value_counts()

In [ ]:
combined_table['Metric'].value_counts()

In [ ]:
# Pivot the table to wide format
wide_format_df = combined_table.pivot_table(
    index=["Year", "Demographic_Category", "Traffic source"],  # Columns to keep as indices
    columns="Metric",  # Column values to spread across
    values="Value",  # Values to populate in the new columns
    aggfunc="first",  # Use first value in case of duplicates (adjust as needed)
).reset_index()

# Fix column names after pivoting
wide_format_df.columns.name = None  # Remove the name of the columns axis
wide_format_df = wide_format_df.rename_axis(None, axis=1)  # Clean up the index

In [ ]:
wide_format_df.info()

In [ ]:
wide_format_df.head()

In [ ]:
wide_format_df.to_excel('v4_case_study_2_combined_table.xlsx', index=False)